
## M161 first question notebook
    - created venv python environment
    - shloud create requirements for resusability
    - data .csv files to large to upload to github will resolve thaht later
## Data cleaning I
 1.  check types 
 2. check for null values


In [18]:
import pandas as pd
file_path = 'bigdata2025classification/train.csv'

def load_and_process_data(file_path):
    # Load data from a CSV file
    dataTrain = pd.read_csv(file_path)

    print("Data loaded successfully.")
    print("First 5 rows of the dataset:")
    print(dataTrain.head())

    print("\nData summary:")
    print(dataTrain.info())

    # Check for missing values in the dataframe
    print("\nMissing values in each column:")
    print(dataTrain.isnull().sum())
    
    return dataTrain

dataTrain = load_and_process_data(file_path)

# check column data types
def check_column_types(dataTrain):
    print("\nColumn data types:")
    print(dataTrain.dtypes)

check_column_types(dataTrain)





Data loaded successfully.
First 5 rows of the dataset:
       Id                                              Title  \
0  227464  Netflix is coming to cable boxes, and Amazon i...   
1  244074  Pharrell, Iranian President React to Tehran 'H...   
2   60707                    Wildlife service seeks comments   
3   27883  Facebook teams up with Storyful to launch 'FB ...   
4  169596           Caesars plans US$880 mln New York casino   

                                             Content          Label  
0   if you subscribe to one of three rinky-dink (...  Entertainment  
1   pharrell, iranian president react to tehran '...  Entertainment  
2   the u.s. fish and wildlife service has reopen...     Technology  
3   the very nature of social media means it is o...     Technology  
4   caesars plans us$880 mln new york casino jul ...       Business  

Data summary:
<class 'pandas.DataFrame'>
RangeIndex: 111795 entries, 0 to 111794
Data columns (total 4 columns):
 #   Column   Non-Null Cou

In [19]:
# Keep only the first 6000 instances for faster experimentation
dataTrain = dataTrain.iloc[:6000].reset_index(drop=True)
print(f"Subset shape: {dataTrain.shape}")

Subset shape: (6000, 4)


## Data cleaning II continue
3. check for duplicates
***************************
### note
- the data types of all exeprt Id column is "object" in pandas, it works, but could be converted to String for a performance uplift.
*****************************

## Remove duplicates

In [20]:
# Remove duplicates based on 'Title' and 'Content' columns, keeping the first occurrence
dataTrain = dataTrain.drop_duplicates(subset=['Title', 'Content'], keep='first')
print("\nDuplicates based on Title and Content removed. Data shape:", dataTrain.shape)


# Reset the index after removing duplicates
dataTrain = dataTrain.reset_index(drop=True)
print("\nIndex reset. Data shape:", dataTrain.shape)
dataTrain.info()


Duplicates based on Title and Content removed. Data shape: (5996, 4)

Index reset. Data shape: (5996, 4)
<class 'pandas.DataFrame'>
RangeIndex: 5996 entries, 0 to 5995
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   Id       5996 non-null   int64
 1   Title    5996 non-null   str  
 2   Content  5996 non-null   str  
 3   Label    5996 non-null   str  
dtypes: int64(1), str(3)
memory usage: 187.5 KB



### **fixed**
*******************

In [21]:
import numpy as np
dataTrain['text_length'] = dataTrain['Content'].apply(len)
dataTrain['word_count'] = dataTrain['Content'].apply(lambda x: len(str(x).split()))
dataTrain['sentence_count'] = dataTrain['Content'].apply(lambda x: len(str(x).split('.')))
dataTrain['avg_word_length'] = dataTrain['Content'].apply(lambda x: np.mean([len(word) for word in str(x).split()]))

print("\n--- Content Statistics ---")
print(dataTrain[['text_length', 'word_count', 'sentence_count', 'avg_word_length']].describe())


--- Content Statistics ---
        text_length   word_count  sentence_count  avg_word_length
count   5996.000000  5996.000000     5996.000000      5996.000000
mean    2560.841561   422.743829       24.667612         5.065045
std     2107.402479   350.217259       24.484260         0.363527
min       17.000000     3.000000        1.000000         3.953704
25%     1293.750000   214.000000       12.000000         4.834019
50%     2023.000000   336.000000       19.000000         5.055181
75%     3154.000000   517.000000       30.000000         5.269693
max    43621.000000  7044.000000      846.000000        12.540541



### Remove words not in English dictionary

- **probably could change dictionary for better results but it works...**


In [22]:
import re
import nltk
from nltk.corpus import words

# Download the words corpus if not already present
nltk.download('words')
english_words = set(words.words())

def clean_text(text):
    # Split text into words
    word_list = re.findall(r'\b\w+\b', str(text))
    cleaned_words = []
    for word in word_list:
        # Drop any word not in dictionary
        if word.lower() not in english_words:
            continue
        # Drop words with 2+ repeating chars not in dictionary (redundant now, but kept for clarity)
        if re.search(r'(.)\1{1,}', word):
            if word.lower() not in english_words:
                continue
        cleaned_words.append(word)
    return ' '.join(cleaned_words)

# Apply to both columns
dataTrain['Title'] = dataTrain['Title'].apply(clean_text)
dataTrain['Content'] = dataTrain['Content'].apply(clean_text)


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\odys_\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


**NOTE**
- TAKES A LOT OF TIME TO EXECUTE ~7 min
*********************

In [23]:
import re
import nltk
import contractions
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer

# Download required NLTK data if not already present
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def clean_text(text):
    # Expand contractions
    text = contractions.fix(text)
    # Convert to lowercase
    text = text.lower()
    # Remove special characters (keep only letters and spaces)
    text = re.sub(r'[^a-z\s]', '', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Tokenize
    words = text.split()
    # Remove stopwords, lemmatize, and stem
    words = [stemmer.stem(lemmatizer.lemmatize(word)) for word in words if word not in stop_words]
    text = ' '.join(words)
    return text

for col in ['Title', 'Content']:
    dataTrain[col] = dataTrain[col].astype(str).apply(clean_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\odys_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\odys_\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\odys_\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [24]:
print(dataTrain.head())

       Id                       Title  \
0  227464  come cabl groceri overlord   
1  244074          presid react happi   
2   60707              wildlif servic   
3   27883                      launch   
4  169596           u new york casino   

                                             Content          Label  \
0  subscrib one three dink compar speak cabl abl ...  Entertainment   
1  presid react happi singer presid took twitter ...  Entertainment   
2  fish wildlif servic comment period addit day p...     Technology   
3  natur social medium often sourc real time brea...     Technology   
4  u new york casino latest news top deck world e...       Business   

   text_length  word_count  sentence_count  avg_word_length  
0         1576         264              15         4.965909  
1         1200         192              10         5.250000  
2         2773         416              34         5.665865  
3         1564         254              13         5.157480  
4         2250  

## Starting future extraction (converting text to numbers for ML algorythms to run)





In [25]:
# Combine 'Title' and 'Content' columns into a single string
# and vectorize the result for classification
from sklearn.feature_extraction.text import CountVectorizer

# Create a new column that combines Title and Content
# (if either column is missing, fill with empty string)
dataTrain['Combined'] = dataTrain['Title'].fillna('') + ' ' + dataTrain['Content'].fillna('')

# Initialize CountVectorizer (Bag of Words)
bow_vectorizer = CountVectorizer(binary=True, max_features=5000, ngram_range=(1,2))  # Use binary=True for presence/absence of words, limit to top 5000 features no limit shape (111220,27139)

# Fit and transform the combined column
dataTrain_bow = bow_vectorizer.fit_transform(dataTrain['Combined'])

# Show shape and a sample
print('Bag of Words matrix shape:', dataTrain_bow.shape)
print("dataTrain_bow sample (first 10 rows):", dataTrain_bow[:10].toarray())
print('Feature names (first 20):', bow_vectorizer.get_feature_names_out()[:20])

Bag of Words matrix shape: (5996, 5000)
dataTrain_bow sample (first 10 rows): [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Feature names (first 20): ['abandon' 'abdomin' 'abil' 'abl' 'abl get' 'abl see' 'abl use' 'aboard'
 'abroad' 'absenc' 'absent' 'absolut' 'absorb' 'absurd' 'abund' 'abus'
 'academ' 'academi' 'academi award' 'acceler']


In [26]:
print('Feature names (first 100):', bow_vectorizer.get_feature_names_out()[:1000])

Feature names (first 100): ['abandon' 'abdomin' 'abil' 'abl' 'abl get' 'abl see' 'abl use' 'aboard'
 'abroad' 'absenc' 'absent' 'absolut' 'absorb' 'absurd' 'abund' 'abus'
 'academ' 'academi' 'academi award' 'acceler' 'accept' 'access' 'accid'
 'accident' 'accommod' 'accomplish' 'accord' 'accord compani'
 'accord data' 'accord diseas' 'accord nation' 'accord new' 'accord news'
 'accord recent' 'accord report' 'accord statement' 'accord studi'
 'accord world' 'accordingli' 'account' 'accur' 'accuraci' 'accus'
 'achiev' 'acknowledg' 'acquir' 'acquisit' 'across' 'across countri'
 'across world' 'act' 'action' 'activ' 'activist' 'actor' 'actress'
 'actual' 'acut' 'ad' 'ad new' 'adapt' 'add' 'addict' 'addit' 'address'
 'adequ' 'adjust' 'administr' 'administr said' 'admir' 'admiss' 'admit'
 'adopt' 'ador' 'adult' 'advanc' 'advantag' 'adventur' 'advers' 'advertis'
 'advertis continu' 'advic' 'advis' 'advisori' 'advocaci' 'aesthet'
 'affair' 'affect' 'affili' 'afford' 'afford care' 'afraid' 'af

## Logistic Regression Classifier

Now we will train a Logistic Regression classifier using the Bag of Words features extracted above. We will evaluate its performance using accuracy, precision, recall, and F1-score.

### 5-Fold Cross-Validation

We will now use 5-fold cross-validation to evaluate the Logistic Regression classifier, instead of a single train/test split. This provides a more robust estimate of model performance.

In [27]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# # Assume the target column is named 'Label' (change if needed)
# if 'Label' not in dataTrain.columns:
#     print("ERROR: 'Label' column not found in dataTrain. Please check your dataset.")
# else:
#     # Split data into features and target
#     X = dataTrain_bow
#     y = dataTrain['Label']

#     # Split into train and test sets
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#     # Initialize and train Logistic Regression
#     clf = LogisticRegression(max_iter=1000, random_state=42)
#     clf.fit(X_train, y_train)

#     # Predict on test set
#     y_pred = clf.predict(X_test)

#     # Evaluate
#     print("Accuracy:", accuracy_score(y_test, y_pred))
#     print("Precision:", precision_score(y_test, y_pred, average='weighted', zero_division=0))
#     print("Recall:", recall_score(y_test, y_pred, average='weighted', zero_division=0))
#     print("F1 Score:", f1_score(y_test, y_pred, average='weighted', zero_division=0))
#     print("\nClassification Report:\n", classification_report(y_test, y_pred, zero_division=0))

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import classification_report

# Assume the target column is named 'Label' (change if needed)
if 'Label' not in dataTrain.columns:
    print("ERROR: 'Label' column not found in dataTrain. Please check your dataset.")
else:
    X = dataTrain_bow
    y = dataTrain['Label']
    clf = LogisticRegression(max_iter=3000,class_weight='balanced',random_state=42)

    # 5-fold cross-validation scores
    accuracy_scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
    print("5-Fold Cross-Validation Accuracy Scores:", accuracy_scores)
    print("Mean Accuracy:", accuracy_scores.mean())
    print("Std Dev:", accuracy_scores.std())

    # Cross-validated predictions for full classification report
    y_pred_cv = cross_val_predict(clf, X, y, cv=5)
    print("\nClassification Report (5-fold CV):\n", classification_report(y, y_pred_cv, zero_division=0))

5-Fold Cross-Validation Accuracy Scores: [0.89416667 0.88907423 0.89908257 0.88573812 0.89574646]
Mean Accuracy: 0.8927616068946345
Std Dev: 0.0047699951622402585

Classification Report (5-fold CV):
                precision    recall  f1-score   support

     Business       0.84      0.84      0.84      1335
Entertainment       0.94      0.95      0.94      2407
       Health       0.92      0.89      0.91       636
   Technology       0.86      0.85      0.85      1618

     accuracy                           0.89      5996
    macro avg       0.89      0.88      0.89      5996
 weighted avg       0.89      0.89      0.89      5996

